In [1]:
import vcf
import numpy as np
import pandas as pd
from pyVEP import VEP
import requests
import json

In [2]:
info_type = {'del': "deletion",
              'ins': "insertion",
              'complex': "complex",
              'snp' : "Single Nucleotide Polymorphism",
              'mnp': "Multi Nucleotide Polymorphism",
              }
variation_type = {'indel' : "INDEL",
              'snp' : "SNP",
              }
variation_subtype = {'del': "deletion",
              'ins': "insertion",
              'ts': "transition",
              'tv': "transversion",
              'unknown': "unknown"}


In [3]:
def parse_vcf_record(record):
    ref_reads, variant_reads, total_reads = record.INFO['RO'], record.INFO['AO'], record.INFO['DP']
                
    var_per = []
    for var_val in variant_reads:
        var_per.append(round((float(var_val)/float(total_reads))*100.0,2))
                    
    ref_per = round((float(ref_reads)/float(total_reads))*100.0,2)
                
    info_types = []
    for infotype in record.INFO['TYPE']:
        info_types.append(info_type[infotype])
                    
    tmp = (record.CHROM, record.POS, record.REF, str(record.ALT)[1:-1],
           variation_type[record.var_type],str(info_types)[1:-1],variation_subtype[record.var_subtype],
           str(variant_reads)[1:-1], total_reads, str(var_per)+"%|"+str(ref_per)+"%",)
                  
    return(tmp) 

In [4]:
def variant_effect(record): 
    
    var_effect = []
    
    for eachalt in record.ALT:
        mystr = str(record.CHROM)+" "+str(record.POS)+" . "+str(record.REF)+" "+str(eachalt)+" .  .  ."
        
        if len(record.REF)>1 and len(eachalt)>1:
            var_effect.append("complex")
        else:
            r = VEP(mystr, 'grch38')
            var_effect.append(str(r[0]['most_severe_consequence']))
           
    return((str(var_effect)[1:-1],))

In [5]:
def ExAC_info(record):
    chrom,pos,ref,alt = record.CHROM, record.POS, record.REF, record.ALT
    tmp = (0,0,0,0,0,"NA","NA")
    
    if len(ref)==1 and len(alt)==1:
        exac_url = "http://exac.hms.harvard.edu/rest/variant/variant/"+chrom+'-'+str(pos)+'-'+ref+'-'+str(alt[0])
        exac_response = requests.get(exac_url)
        
        if (exac_response.status_code == 200) :
            exac_response_json = exac_response.json()
    
            allele_count = exac_response_json["allele_count"]
            allele_num = exac_response_json["allele_num"]
            allele_freq = exac_response_json["allele_freq"]
            num_homozygotes = exac_response_json["hom_count"]
            site_quality = exac_response_json["site_quality"]
            outFilter = exac_response_json["filter"]
    
            major_consequence={}
            vep_ann = exac_response_json["vep_annotations"]   
            for vep in vep_ann:
                major_consequence[vep["HGVSc"]]=vep["major_consequence"]
            
            tmp = (allele_count,allele_num,allele_freq,num_homozygotes,site_quality,str(outFilter),str(json.dumps(major_consequence))[1:-1])
        else:
            raise ValueError("ERROR: ExAC response status_code should be 200, but it is "+exac_response.status_code+" !")
        
    return(tmp)
    

In [6]:
def main(vcf_file):
    
    vcf_reader = vcf.Reader(open(vcf_file,'r'))
    #snv_dtype = [('chromosome','S50'),('position',int),('reference','S50'),('variant','S50'),
    #             ('var_type','S50'),('var_infotype','S50'),('var_subtype','S50'),
    #             ('var_count','S50'),('read_depth',int),('var%|ref%','S50'),
    #             ('var_effect','S50'),
    #             ('ExAC_allele_count',int),('ExAC_allele_num',int),('ExAC_allele_freq',float),
    #             ('ExAC_homozygotes_num',int),('ExAC_site_quality',float),
    #             ('ExAC_filter','S50'),('ExAC_HGVSc:major_consequence','S50')]
    
    #snv_df = np.empty([0,18],dtype=snv_dtype)
    f = open("./vcf-parse-challenge-output.txt","w+")
    f.write('chromosome'+'\t'+'position'+'\t'+'reference'+'\t'+'variant'+'\t'+'var_type'+'\t'+'var_infotype'+'\t'+
            'var_subtype'+'\t'+'var_count'+'\t'+'read_depth'+'\t'+'var%|ref%'+'\t'+'var_effect'+'\t'+
            'ExAC_allele_count'+'\t'+'ExAC_allele_num'+'\t'+'ExAC_allele_freq'+'\t'+'ExAC_homozygotes_num'+'\t'+
            'ExAC_site_quality'+'\t'+'ExAC_filter'+'\t'+'ExAC_HGVSc:major_consequence'+'\n')
    
    for record in vcf_reader:
        try:
            #if record.var_type == 'indel' and record.var_subtype == 'unknown' :  
            #if record.var_type == 'snp':
                vcf_out = parse_vcf_record(record)
                eff_out = variant_effect(record)
                try:
                    ExAC_out = ExAC_info(record)
                except KeyError:
                    ExAC_out = (0,0,0,0,0,"NA","NA")
                #snv_df = np.append(snv_df,np.array(vcf_out+eff_out+ExAC_out, dtype=snv_dtype))
                out_line = vcf_out+eff_out+ExAC_out
                f.write('\t'.join(str(i) for i in out_line)+'\n')
        except KeyError:
            print('WARNING: missing count field(s) in record %s:%d' % (record.CHROM, record.POS))

    f.close()
    #return pd.DataFrame(snv_df) 
    return()

In [ ]:
main('./Challenge_data.vcf')

In [ ]:
output = main('./Challenge_data.vcf')
print(output.iloc[0:10,:])
f= open("./vcf-parse-challenge-output.txt","w+")
f.write(output)
f.close()